# ECE 763 Project 2: Babysitting the CNN Training Process
## Nathan Starliper
## 4/9/2019

### Introduction

In this project we use the CelebA dataset. The first 11000 images are taken from the dataset which will be split into 10000 training images and 1000 testing images. The images are resized to 60x60. We also extract 60x60 background image patches to be used as our "non-face" class. The image extraction is done before hand using the *extract_images.py* script. The *utils.py* script contains the code for loading the images and splitting in training, validation, and testing sets. It also contains the function for preprocessing the data using standardization. The "cnn.py" script contains the functions for building, training, and testing the model. The *cross_val.py* script contains the code for running cross-validation hyperparameter search for learning rate and dropout. Finally, the *run_cnn.py* script contains the code to run a full 50-epoch training and final testing of the model. This is the script that should be run to recreate the final results at the end. The anaconda environment file has also been provided to easily setup the environment used to create this code and install all necessary dependencies, this file is stored in *env/environment.yml*. The environment can be created from the *.yml* file in bash using: 
```sh
conda env create -f environment.yml
```

### Model

Here we construct a CNN modeled somewhat similiarly to LeNet. It contains two convolutional layers with ReLU activations and each followed by a max pooling layer. This is followed by a dense layer and softmax activation. We also add a dropout layer after the dense layer for regularization.
<img src="files/report_images/network_parms.png">

### Preprocessing

First we preprocess the data by subtracting the training mean and dividing by the training standard deviation (this is known as standardizing). 

```python
def standard(train_img, val_img, test_img):
    train_mean = np.mean(train_img, axis=0)
    train_std = np.std(train_img, axis=0)
    train_img_std = np.reshape(np.divide((train_img - train_mean), train_std), (len(train_img), 60, 60, 3))
    valid_img_std = np.reshape(np.divide((val_img - train_mean), train_std), (len(val_img), 60, 60, 3))
    test_img_std = np.reshape(np.divide((test_img - train_mean), train_std), (len(test_img), 60, 60, 3))
    return train_img_std, valid_img_std, test_img_std
```

### Training the Model

Let's run a single run of our network for a sanity check on our intial loss:

```python
model = cnn.build_model(input_shape, activation_fn, learn_rate, drop_prob, num_neurons, num_classes)
train_accuracy, train_loss, valid_accuracy, valid_loss = cnn.train_model(model, train_images[0], train_labels[0], 
                                                                         batch_size,num_epochs, valid_images[0], 
                                                                         valid_labels[0], tb_callback)
```
<img src="files/report_images/loss_sanity_check.png">

These results look good, a loss of 0.2607 with a random correct guess makes sense. 

Now we will increase regularization (dropout) to 0.5

<img src="files/report_images/loss_check_regular.png">

The loss goes up as we expect.

Now let's make sure that we can overfit a small set of the training data.

```python
train_accuracy, train_loss, valid_accuracy, valid_loss = cnn.train_model(model, train_images[:20], train_labels[:20],
                                                                         batch_size,num_epochs, valid_images[:20],
                                                                         valid_labels[:20], tb_callback)
```
<table><tr><td><img src="files/report_images/small_data1.png"></td><td><img src="files/report_images/small_data2.png"></td></tr></table>

Very low loss and 100% training accuracy with lower validation accuracy
, successful overfit!

Let's set the dropout low and set the learning rate to 1e-6: 

```python
num_epochs = 10
learn_rate = 1e-6
drop_prob = 0.01
```

<img src="files/report_images/loss_lowlearn.png">

The loss is decreasing very slowly, the learning rate is way too low. Let's try a much higher rate: 1e6. 

<img src="files/report_images/loss_highlearn.png">

The losses are both very high and are not changing at all. This means the learning rate is way too high.

Let's do some hyperparameter tuning on the dropout and learning rate. We will start with very course parameter spaces and narrow it down from there:
```python
num_epochs = 5
max_count = 100
for count in range(0, max_count):
    learn_rate = 10**uniform(-3, -6)
    drop_prob = 10**uniform(-4, 0)
```
<img src="files/report_images/hyper_course.png">

This gives us a better idea of what kind range of parameters we should use. One thing we noticed here is that our optimal value for learning rate is actually right on the higher border of our search space, which tells us we should update our upper bound. Now we will use a smaller parameter space to do finer tuning:
```python
num_epochs = 5
max_count = 50
for count in range(0, max_count):
    learn_rate = 10**uniform(-2, -4)
    drop_prob = 10**uniform(-2, 0)
```

<img src="files/report_images/hyper_fine.png">

Analyzing the results we can see that the optimal learning rate is 0.007 and optimal dropout is around 0.1.

Now we will use our optimal learning rate and dropout to find the best optimizer. Let's try SGD, RMSProp, and Adam. First let's see the results of SGD:
<table><tr><td><img src="files/report_images/SGD_loss.png"></td><td><img src="files/report_images/SGD_acc.png"></td></tr></table>
<img src="files/report_images/SGD_final.png">

These results look pretty good. Let's see how they compare to RMSprop

Now we will try with RMSprop. After the initial RMSProp run, it was determined that the optimal learning rate for SGD of 0.007 was actually too high for RMSprop and caused the losses to explode. The learning rate was then reduced to 0.001 and the results are as follows:

<table><tr><td><img src="files/report_images/RMSprop_loss.png"></td><td><img src="files/report_images/RMSprop_acc.png"></td></tr></table> 
<img src="files/report_images/rmsprop_final.png">

It looks like RMSProp did a little bit better than SGD. 

Finally, let's see the results of using the Adam optimizer:

<table><tr><td><img src="files/report_images/Adam_loss.png"></td><td><img src="files/report_images/Adam_acc.png"></td></tr></table>
<img src="files/report_images/adam_final.png">

Looks like Adam gives us the best results.

Now just for the sake of comparison let's see how the training looks without preprocessing of the data:

<img src="files/report_images/no_pre.png">

Clearly, without preprocessing the data our gradient is exploding due to the large feature values. This shows us how necessary it is to preprocess our images before training the network.

Let's do a full 50 epoch training and validation using the Adam optimizer.
<img src="files/report_images/Adam_loss_nodecay.png">

These are not the best results, it looks like performance begins to degrade over time (most likely overfitting). Let's try to add in learning rate decay. We set the decay rate to 0.001. 

Let's do a full 50 epoch training and validation using the Adam optimizer with learning rate decay 0.001:
<table><tr><td><img src="files/report_images/Adam_loss_full.png"></td><td><img src="files/report_images/Adam_acc_full.png"></td></tr></table>
<img src="files/report_images/adam_final_full.png">

This looks much better. 

### Testing the Model

Now let's test our trained model using the unseen test datset:

<img src="files/report_images/adam_final_test.png">

That's some really great accuracy for only 50 epochs of training. 